# Conceptual description

As people interact, they tend to become more alike in their beliefs, attitudes and behaviour. In ["The Dissemination of Culture: A Model with Local Convergence and Global Polarization" (1997)](https://www.jstor.org/stable/174371?seq=1#metadata_info_tab_contents), Robert Axelrod presents an agent-based model to explain cultural diffusion. Analogous to Schelling's segregation model, the key to this conceptualization is the emergence of polarization from the interaction of individual agents. The basic premise is that the more similar an agent is to a neighbor, the more likely that that actor will adopt one of the neighbor's traits.

In the model below, this is implemented by initializing the model by filling an excel-like grid with agents with random values [0,1] for each of four traits (music, sports, favorite color and drink). 

Each step, each agent (in random order) chooses a random neighbor from its neighbors proportionaly to how similar it is to each of its neighbors, and adopts one randomly selected differing trait from this neighbor. Similarity between any two agents is calculated by the sum of identical traits.

To visualize the model, the four traits are transformed into 'RGBA' (Red-Green-Blue-Alpha) values; i.e. a color and an opacity. The visualizations below show the clusters of homogeneity being formed.

## implement the step method of the agent
The first assignment is to implement the step method for the agent. 

1. Make a conceptual description or diagram of the step method for the agent in light of the description given above.
2. implement the step method.
    * you need to use self.cell.get_neighborhood in order to make it work with varying neighborhood sizes
    * self.random.choices takes an optional weights keyword argument allowing you to randomly yet proportionaly to similiary select a neighbor

Compare your results with those shown below
<div>
<img src="dynamics.png" width="600"/>
</div>

In [ ]:
import collections
import random

import numpy as np

from mesa import Model
from mesa.datacollection import DataCollector

from mesa.experimental.cell_space import CellAgent, OrthogonalMooreGrid


class CulturalDiff(Model):
    """
    Model class for the Schelling segregation model.
    
    Parameters
    ----------
    height : int
             height of grid
    width : int
            height of grid
    seed : int
            random seed
    neighborhoood_size : int, optional
    torus : bool, optional
    
    Attributes
    ----------
    height : int
    width : int
    density : float
    schedule : RandomActivation instance
    grid : SingleGrid instance
    
    """


    def __init__(self, height=20, width=20, seed=None,
                neighborhood_size=1, torus=True):
        super().__init__(seed=seed)

        self.grid = OrthogonalMooreGrid((width, height), torus=torus, capacity=1, random=self.random)
        self.datacollector = DataCollector(model_reporters={'diversity':calculate_nr_of_cultures})

        # Fill grid with agents with random traits
        
        # Note that this implementation does not guarantee some set distribution of traits. 
        # Therefore, examining the effect of minorities etc is not facilitated.

        for cell in self.grid.all_cells:
            profile = np.asarray([self.random.choice([0,1]) for _ in range(4)])
            
            agent = CulturalDiffAgent(self, profile, neighborhood_size)
            agent.cell = cell

        self.datacollector.collect(self)

    def step(self):
        """
        Run one step of the model.
        """
        self.agents.shuffle_do("step")
        self.datacollector.collect(self)


class CulturalDiffAgent(CellAgent):
    """
    Schelling segregation agent
    
    Parameters
    ----------
    model : Model instance
    profile : ndarray
    
    """

    def __init__(self, model, profile, neighborhood_size):
        super().__init__(model)
        self.profile = profile
        self.neighborhood_size = neighborhood_size
        
    def step(self):
        ...


def traits_to_color(profile):  
    """ Converts the traits of an agent to a list of RGBA values"""
    color = profile.copy().astype(float)
    if color[-1]==0:
        color[-1] = 0.2
    return color


def calculate_nr_of_cultures(model):
    diversity = collections.defaultdict(int)
    for agent in model.agents:
        diversity[tuple(agent.profile)] += 1

    return len(diversity.keys())

In [ ]:
# Create initial model instance
model = CulturalDiff(50, 50)

for _ in range(10):
    model.step()

# Visualization

In [ ]:
from mesa.visualization import (
    SolaraViz,
    make_plot_component,
    make_space_component,
)
from mesa.visualization.components import AgentPortrayalStyle

In [ ]:
def agent_portrayal(agent):
    return AgentPortrayalStyle(size=25, color=traits_to_color(agent.profile), marker='s')

model_params = {
    "neighborhood_size": {
        "type": "SliderInt",
        "value": 1,
        "label": "neighborhood size",
        "min": 1,
        "max": 4,
        "step": 1,
    },
    "width": 20,
    "height": 20,
}

# Create initial model instance
model = CulturalDiff(50, 50)

SpaceGraph = make_space_component(agent_portrayal)
DiversityPlot = make_plot_component("diversity")

page = SolaraViz(
    model,
    components=[SpaceGraph, DiversityPlot],
    model_params=model_params,
    name="Cultural diffusion",
)
# This is required to render the visualization in the Jupyter notebook
page


# Neighborhood size

The original version of the model uses a Moore neighborhood with a radius of 1. Explore the dynamics of the model with radius ranging from 1-3. How do the dynamics of the model change, and why?

1. Perform experiments for neighborhoods with radiuses ranging from 1-3. Don't forget to account for the stochastic uncertainty
2. Make an appropriate visualization showing the difference in dynamics over time for both versions of the model
3. Describe in words how the behavior is different and explain this change in behavior.

# Torus
To deal with edge effects, the model assumes the use of a Torus (i.e., donut). In reality, geography matters a lot and thus edge effects can occur. Explore the dynamics of the model with and without assuming a torus. How does the behaviour of the model change, and why?

1. Perform experiments to test the behavior of the model with and without assuming a torus. Don't forget to account for the stochastic uncertainty
2. Make an appropriate visualization showing the difference in dynamics over time for both versions of the model
Describe in words how the behavior is different and explain this change in behavior.

# Neighbor interaction
The provided implementation assumes that interaction is random yet proportional to how similar the neighbors are. What if instead, we assume completely random interaction? How does the behavior of the model change, and why?

Note that in order to do this, you will have to change the existing implementation:

* add a select_random keyword argument to the mode
* set select_random as a class attribute on the Agent class
* add a select_random_neigbor and select_weighted_random_neigbor method to the Agent class
* pick which select_* method to use based on the value of the select_random class attribute

1. Implement the modified model and agent
2. Perform experiments for both random and weighted random selection of neighbors. Don't forget to account for the stochastic uncertainty
3. Make an appropriate visualization showing the difference in dynamics over time for both versions of the model
4. Describe in words how the behavior is different and explain this change in behavior.
5. Conceptually, how would you change the original implementation of the model as used for the first assignment such that the extensions needed for this assignment could have been implemented by merely extending the model and agent class.
